## Promoter

In [101]:
import pandas as pd
# 读取CSV文件
df = pd.read_csv('/home/ykq/projects/CPSC0926/mouse/all_dna.csv')
promoter_df = df[df['wghmm.state.type'] == 'Promoter'].copy()

promoter_df['center'] = (promoter_df['wghmm.start'] + promoter_df['wghmm.end']) / 2
promoter_df['start'] = promoter_df['center'].astype(int) - 1000
promoter_df['end'] = promoter_df['center'].astype(int) + 1000

# 创建newID
promoter_df['newID'] = promoter_df['wghmm.chr'] + '-' + promoter_df['start'].astype(str) + '-' + promoter_df['end'].astype(str)
target_strings = promoter_df['newID'].tolist()

# 读取另一个文件并筛选
df2 = pd.read_csv('/home/ykq/projects/CPSC0926/mouse/traindata.csv')
filtered_df2 = df2[df2['ID'].isin(target_strings)]
os.makedirs('/home/ykq/projects/CPSC0926/mouse/train', exist_ok=True)
os.makedirs('/home/ykq/projects/CPSC0926/mouse/train/P', exist_ok=True)
# 保存筛选结果
filtered_df2.to_csv('/home/ykq/projects/CPSC0926/mouse/train/P/traindata.csv', index=False)

## Labeled training data

In [103]:
import pandas as pd
# 读取数据
p_dna = pd.read_csv('/home/ykq/projects/CPSC0926/mouse/all_dna.csv')
train_data = pd.read_csv('/home/ykq/projects/CPSC0926/mouse/train/P/traindata.csv')

'''
方法一：按照baseMean的20%取cutoff，HCP和MCP做正样本，LCP做负样本
方法二：按照baseMean的20%取cutoff，HCP做正样本，LCP和MCP做负样本
方法三：按照baseMean的20%取cutoff，HCP做正样本，LCP做负样本,观察MCP的分布
'''

# 方法一
# filtered_p_dna_1 = p_dna[(p_dna['pstype'].isin(['HCP', 'MCP'])) & (p_dna['baseMean'] >= p_dna['baseMean'].quantile(0.2))].copy()
# filtered_p_dna_2 = p_dna[(p_dna['pstype'] == 'LCP') & (p_dna['baseMean'] >= p_dna['baseMean'].quantile(0.2))].copy()

# 方法二
# filtered_p_dna_1 = p_dna[(p_dna['pstype'].isin(['HCP'])) & (p_dna['baseMean'] >= p_dna['baseMean'].quantile(0.2))].copy()
# filtered_p_dna_2 = p_dna[(p_dna['pstype'].isin(['LCP', 'MCP'])) & (p_dna['baseMean'] >= p_dna['baseMean'].quantile(0.2))].copy()

# 方法三
filtered_p_dna_1 = p_dna[(p_dna['pstype'].isin(['HCP']))].copy()
filtered_p_dna_2 = p_dna[(p_dna['pstype'].isin(['LCP']))].copy()

# 处理条件1的数据
filtered_p_dna_1.loc[:, 'center'] = (filtered_p_dna_1['wghmm.start'] + filtered_p_dna_1['wghmm.end']) / 2
filtered_p_dna_1.loc[:, 'start'] = filtered_p_dna_1['center'].astype(int) - 1000
filtered_p_dna_1.loc[:, 'end'] = filtered_p_dna_1['center'].astype(int) + 1000
filtered_p_dna_1.loc[:, 'newID'] = filtered_p_dna_1['wghmm.chr']+'-'+filtered_p_dna_1['start'].astype(str)+'-'+filtered_p_dna_1['end'].astype(str)

# 处理条件2的数据
filtered_p_dna_2.loc[:, 'center'] = (filtered_p_dna_2['wghmm.start'] + filtered_p_dna_2['wghmm.end']) / 2
filtered_p_dna_2.loc[:, 'start'] = filtered_p_dna_2['center'].astype(int) - 1000
filtered_p_dna_2.loc[:, 'end'] = filtered_p_dna_2['center'].astype(int) + 1000
filtered_p_dna_2.loc[:, 'newID'] = filtered_p_dna_2['wghmm.chr']+'-'+filtered_p_dna_2['start'].astype(str)+'-'+filtered_p_dna_2['end'].astype(str)

# 获取两种条件下的目标字符串
target_strings_1 = filtered_p_dna_1['newID'].tolist()
target_strings_2 = filtered_p_dna_2['newID'].tolist()

# 修改traindata.csv文件中Y列的值
train_data.loc[
    (train_data['chromosome'].isin(filtered_p_dna_1['wghmm.chr'])) &
    (train_data['start'].isin(filtered_p_dna_1['start'])) &
    (train_data['end'].isin(filtered_p_dna_1['end'])) &
    (train_data['ID'].isin(target_strings_1)),
    'Y'
] = 1

train_data.loc[
    (train_data['chromosome'].isin(filtered_p_dna_2['wghmm.chr'])) &
    (train_data['start'].isin(filtered_p_dna_2['start'])) &
    (train_data['end'].isin(filtered_p_dna_2['end'])) &
    (train_data['ID'].isin(target_strings_2)),
    'Y'
] = 2

# 保存修改后的文件
train_data.to_csv('/home/ykq/projects/CPSC0926/mouse/train/P/traindata_withlabel1.csv', index=False)
print('finish')

finish


In [104]:
import pandas as pd
df = pd.read_csv('/home/ykq/projects/CPSC0926/mouse/train/P/traindata_withlabel1.csv')
df_selected = df[df['Y'].isin([1, 2])]
#df_selected = df[df['Y'].isin([0, 1, 2])]
# 创建新文件夹
# os.makedirs('/home/ykq/projects/CPSC/K562/train/traintest/2', exist_ok=True)
df_selected.loc[df_selected['Y'] == 2, 'Y'] = 0

# 保存新的CSV文件
df_selected.to_csv('/home/ykq/projects/CPSC0926/mouse/train/P/traindata_withlabel.csv', index=False)
print('finish')

finish


## Train, valid and test data

In [105]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
change_dir='P'
# 读取数据
file_path =f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata_withlabel.csv'  # 替换为你的文件路径
df = pd.read_csv(file_path)

# 根据 8:1:1 拆分数据
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42)  # 20% 测试集
train_df, val_df = train_test_split(train_val_df, test_size=0.111, random_state=42)  # 25% 测试集 -> 20% 总数据集

# 创建输出目录
output_dir = f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata'
os.makedirs(output_dir, exist_ok=True)

# 保存文件
train_df.to_csv(os.path.join(output_dir, 'train.csv'), index=False)
test_df.to_csv(os.path.join(output_dir, 'test.csv'), index=False)
val_df.to_csv(os.path.join(output_dir, 'val.csv'), index=False)

print("数据处理完成，文件已保存。")

数据处理完成，文件已保存。


## 运行下面这个代码的时候，现在默认是阳性样本点少于阴性样本点，如果报错就交换一下，用注释内的内容

In [107]:
import pandas as pd
import numpy as np
change_dir='P'
def downsample_dataframe(file_path, target_column='Y'):
    df = pd.read_csv(file_path)

    #阳性样本点大于阴性样本点
    df_minority = df[df[target_column] == 0]
    df_majority = df[df[target_column] == 1]
    #阳性样本点小于阴性样本点
    # df_minority = df[df[target_column] == 1]
    # df_majority = df[df[target_column] == 0]
    
    df_majority_downsampled = df_majority.sample(n=len(df_minority), random_state=42)
    df_balanced = pd.concat([df_majority_downsampled, df_minority])

    # --- 6. (重要步骤) 打乱整个数据集的顺序 ---
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    return df_balanced

# --- 主程序入口 ---
if __name__ == '__main__':
    input_file = f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata/train.csv'
    output_file = f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata/equaltrain.csv'
    balanced_df = downsample_dataframe(input_file, target_column='Y')
    balanced_df.to_csv(output_file, index=False)
    print(f"已将平衡后的数据保存到: {output_file}")

已将平衡后的数据保存到: /home/ykq/projects/CPSC0926/mouse/train/P/traindata/equaltrain.csv


In [109]:
import os
import numpy as np
import pandas as pd
change_dir='P'
# 参数配置
# 单细胞数据
input_dir = '/home/ykq/projects/CPSC0926/mouse/features/'  # .npy文件存放目录
# bulk数据
# input_dir = '/home/ykq/projects/CPSC/0922/GM12878/features/'  # .npy文件存放目录
labels_csv = f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata_withlabel.csv'  # CSV文件路径
output_dir = f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata'  # 输出路径

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 加载标签数据
labels_df = pd.read_csv(labels_csv)

# 循环处理 train、val 和 test 数据
for split_name in ["equaltrain", "val", "test"]:
    # 读取对应的 CSV 文件
    split_file_path = os.path.join(output_dir, f'{split_name}.csv')
    split_df = pd.read_csv(split_file_path)

    data_list = []
    labels = []

    # 遍历每一行，加载对应的 NPY 数据文件
    for index, row in split_df.iterrows():
        file_id = row['ID']  # 获取文件ID （假设ID列名为 'ID'）
        label = row['Y']  # 获取标签 （假设Y列名为 'Y'）

        # 加载对应的 NPY 文件
        npy_file_path = os.path.join(input_dir, f"{file_id}.npy")
        if os.path.exists(npy_file_path):
            matrix = np.load(npy_file_path)  # 加载 .npy 数据
            data_list.append(matrix)
            labels.append(label)
        else:
            print(f"未找到文件：{npy_file_path}")

    # 转为 numpy 数组
    data_array = np.stack(data_list, axis=0)
    labels_array = np.array(labels)

    # 保存为 .npz 文件
    np.savez(os.path.join(output_dir, f"{split_name}_data.npz"), data=data_array)
    np.savez(os.path.join(output_dir, f"{split_name}_labels.npz"), labels=labels_array)

    print(f"{split_name} 数据 shape={data_array.shape}, 标签 length={len(labels_array)}\n")

equaltrain 数据 shape=(7540, 19, 24), 标签 length=7540

val 数据 shape=(948, 19, 24), 标签 length=948

test 数据 shape=(949, 19, 24), 标签 length=949



## 带ATAC的训练

In [110]:
# -*- coding: utf-8 -*-
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from copy import deepcopy
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, average_precision_score
from Model import ConvAndAttentionModel
import os
import pandas as pd
import matplotlib

matplotlib.use('PDF')
import matplotlib.pyplot as plt
from FocalLoss import FocalLoss
from sklearn.utils import class_weight
import pickle
import seaborn as sns
from sklearn.metrics import confusion_matrix

parameter='alpha5'
change_dir='P'
base_dir=f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata/'
if not os.path.exists(base_dir+'scaler'):
    os.makedirs(base_dir+'scaler', exist_ok=True)
    os.makedirs(base_dir+'picture', exist_ok=True)
    os.makedirs(base_dir+'predict_test', exist_ok=True)

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)  # 转换为 Tensor
        self.labels = torch.tensor(labels, dtype=torch.long)  # 转换为 Tensor (分类)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


def load_data(data_file, labels_file):
    """
    加载 .npz 文件中的数据和标签。
    """
    data = np.load(data_file)['data']
    labels = np.load(labels_file)['labels']
    return data, labels

def process_features_norm(features):
    n_samples, n_timesteps, n_features = features.shape
    # 创建拷贝，避免修改原始数据
    scaled_features = features.copy()
    # print(scaler_metrics)
    for feature_idx in range(n_features):
        if feature_idx == 2:
            # 提取当前特征的所有数据
            # 形状变为 (100 * 19, 1)
            feature_column = scaled_features[:, :, feature_idx].reshape(-1, 1)
            # 取对数
            feature_column = np.log1p(feature_column)

            minmax_scaler = MinMaxScaler(feature_range=(-1, 1))
            normalized_results = minmax_scaler.fit_transform(feature_column)

            scaled_features[:, :, feature_idx] = normalized_results.reshape((n_samples, n_timesteps))

    return scaled_features


def process_features(features):
    n_samples, n_timesteps, n_features = features.shape
    # 创建拷贝
    scaled_features = features.copy()
    scaler_metrics = np.zeros((n_features, 4))
    # print(scaler_metrics)
    for feature_idx in range(n_features):
        # 提取当前特征的所有数据
        feature_column = features[:, :, feature_idx].reshape(-1, 1)
        # 标准化处理
        std_scaler = StandardScaler()
        std_results = std_scaler.fit_transform(feature_column)
        scaler_metrics[feature_idx, 0] = std_scaler.mean_[0]
        scaler_metrics[feature_idx, 1] = std_scaler.scale_[0]
        minmax_scaler = MinMaxScaler(feature_range=(-1, 1))
        normalized_results = minmax_scaler.fit_transform(std_results)
        scaler_metrics[feature_idx, 2] = minmax_scaler.min_[0]
        scaler_metrics[feature_idx, 3] = minmax_scaler.scale_[0]
        scaled_features[:, :, feature_idx] = normalized_results.reshape((n_samples, n_timesteps))
        # 保存
    with open(f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata/scaler/scaler_metrics_V2.pkl', 'wb') as f:
        pickle.dump(scaler_metrics, f)
    return scaled_features


def process_features_predict(features):
    with open(f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata/scaler/scaler_metrics_V2.pkl', 'rb') as f:
        scaler_metrics = pickle.load(f)
    n_samples, n_timesteps, n_features = features.shape
    # 创建拷贝
    scaled_features = features.copy()
    for feature_idx in range(n_features):
        # 提取当前特征的所有数据
        feature_column = scaled_features[:, :, feature_idx].reshape(-1, 1)
        stdmean = scaler_metrics[feature_idx, 0]
        stdscale = scaler_metrics[feature_idx, 1]
        feature_column -= stdmean
        feature_column /= stdscale
        minmaxmin = scaler_metrics[feature_idx, 2]
        minmaxscale = scaler_metrics[feature_idx, 3]
        feature_column *= minmaxscale
        feature_column += minmaxmin
        clipped_data = np.clip(feature_column, -1, 1)
        scaled_features[:, :, feature_idx] = clipped_data.reshape((n_samples, n_timesteps))
    return scaled_features


def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()  # 模型设置为训练模式
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_data, batch_labels in train_loader:
        # 将数据传输到 GPU
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        # 前向传播
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 计算损失和准确率
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
    accuracy = 100.0 * correct / total
    return running_loss / len(train_loader), accuracy

def evaluate(model, val_loader, criterion, device, threshold=0.5, return_preds=False):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    if return_preds:
        all_labels = []
        all_probs = []

    with torch.no_grad():
        for batch_data, batch_labels in val_loader:
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.to(device)
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item()
            probs = torch.softmax(outputs, dim=1)
            predicted = (probs[:, 1] >= threshold).long()
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()

            if return_preds:
                # 注意：这里我们用 .append() 来添加整个批次的数据
                all_labels.append(batch_labels.cpu().numpy())
                all_probs.append(probs.cpu().numpy())

    accuracy = 100.0 * correct / total

    if return_preds:
        # 在函数返回前，直接进行拼接
        final_labels = np.concatenate(all_labels, axis=0)
        final_probs = np.concatenate(all_probs, axis=0)
        return val_loss / len(val_loader), accuracy, final_labels, final_probs
    else:
        return val_loss / len(val_loader), accuracy

    # 根据请求返回不同的结果
    if return_preds:
        return val_loss / len(val_loader), accuracy, all_labels, all_probs
    else:
        return val_loss / len(val_loader), accuracy


def plot_loss_and_accuracy(train_losses, val_losses, train_accuracies, val_accuracies, num_epochs, work_dic):
    """
    绘制损失率和准确率曲线
    """
    epochs = range(1, num_epochs + 1)
    # 创建一个带有损失和准确率的双子图
    plt.figure(figsize=(12, 5))
    # 绘制损失曲线
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss over Epochs')
    plt.legend()
    # 绘制准确率曲线
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Train Accuracy')
    plt.plot(epochs, val_accuracies, label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.ylim(0, 100)  # 限定准确率区间为 0% ~ 100%
    plt.title('Accuracy over Epochs')
    plt.legend()
    # 显示图像
    plt.tight_layout()
    plt.savefig(work_dic + f'picture/ACC{parameter}.pdf', format='pdf')


def plot_roc_curve(labels, probs, work_dic):
    """
    绘制 ROC 曲线并计算 AUC 值
    """
    # 将多分类问题转化为二分类问题（One-vs-Rest）
    fpr, tpr, _ = roc_curve(labels, probs[:, 1])  # 假设正类是第 1 类
    roc_auc = auc(fpr, tpr)
    # 绘制 ROC 曲线
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    plt.savefig(work_dic + f'picture/ROC{parameter}.pdf', format='pdf')


def plot_pr_curve(labels, probs, work_dic):
    """
    绘制 Precision-Recall 曲线并计算 AP 值
    """
    # 假设是二分类问题，取正类概率
    precision, recall, _ = precision_recall_curve(labels, probs[:, 1])
    avg_precision = average_precision_score(labels, probs[:, 1])
    # 绘制 PR 曲线
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, label=f'AP = {avg_precision:.2f}')
    plt.plot([0, 1], [1, 0], 'k--', label='Random Guess')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='lower left')
    plt.savefig(work_dic + f'picture/PR{parameter}.pdf', format='pdf')


work_dic = f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata/'
data_dic = f'/home/ykq/projects/CPSC0926/mouse/train/{change_dir}/traindata/'


def main():
    # 加载训练、验证、测试集
    train_data, train_labels = load_data(data_dic + '/equaltrain_data.npz', data_dic + '/equaltrain_labels.npz')
    val_data, val_labels = load_data(data_dic + '/val_data.npz', data_dic + '/val_labels.npz')
    test_data, test_labels = load_data(data_dic + '/test_data.npz', data_dic + '/test_labels.npz')
    # 先对atac做归一化
    train_data_Norm = process_features_norm(train_data)
    val_data_Norm = process_features_norm(val_data)
    test_data_Norm = process_features_norm(test_data)

    # 处理数据
    train_data_scaled = process_features(train_data_Norm)
    val_data_scaled = process_features_predict(val_data_Norm)
    test_data_scaled = process_features_predict(test_data_Norm)

    # 创建 PyTorch 的 Dataset
    train_dataset = CustomDataset(train_data_scaled, train_labels)
    val_dataset = CustomDataset(val_data_scaled, val_labels)
    test_dataset = CustomDataset(test_data_scaled, test_labels)
    # 创建 Dataloader
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)
    # model_V2
    first_conv_out_channels = 32
    first_kernel_size = (2, 24)
    second_conv_out_channels = 64
    second_kernel_size = (2, 32)
    attention_output_dim = 16  # V 的行数
    maxpool_kernel_size = 16
    model = ConvAndAttentionModel(
        first_conv_out_channels=first_conv_out_channels,
        first_kernel_size=first_kernel_size,
        second_conv_out_channels=second_conv_out_channels,
        second_kernel_size=second_kernel_size,
        attention_output_dim=attention_output_dim,
        maxpool_kernel_size=maxpool_kernel_size)
    # 直接使用labels
    ground_truth = train_labels  # 假设labels已经是类别索引
    # 计算类别权重
    unique_classes = np.unique(ground_truth)
    class_weights = class_weight.compute_class_weight('balanced',
                                                      classes=unique_classes,
                                                      y=ground_truth)
    # # 将模型移动到 GPU（如果可用）
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #
    weights = torch.FloatTensor(class_weights).to(device)
    # 创建 Loss
    #criterion = nn.CrossEntropyLoss()
    # default Parameters: alpha=0.6,gamma=2,lr=0.0005)
    criterion = FocalLoss(alpha=0.5, gamma=2, weight=weights)
    optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
    # 学习率调度器
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    model.to(device)
    num_epochs = 200
    # 用于记录训练过程中的损失和准确率
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    min_val_loss = float('inf')
    best_epoch = 0
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        #scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Epoch {epoch + 1}, Current LR: {current_lr:.6f}')
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        # 记录损失和准确率
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.2f}%")
        # 更新最小测试损失
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            best_epoch = epoch + 1

        #每10个epoch保存一次模型（新增逻辑）
        # if (epoch + 1) % 10 == 0:  # epoch从0开始计数，所以+1
        #     periodic_save_path = f"{data_dic}/model_every10epoch/P_default_V2_epoch{epoch + 1}.pth"
        #     torch.save(model.state_dict(), periodic_save_path)
        #     print(f"Periodic model saved at epoch {epoch + 1}")
    print(f"模型已保存")
    print(f"最小测试损失 {min_val_loss:.4f} 出现在第 {best_epoch} 轮")
    model_save_path = f"{data_dic}/Model_V2_epoch{best_epoch}_{parameter}.pth"
    torch.save(model.state_dict(), model_save_path)


    # 绘制训练过程中的损失和准确率曲线
    plot_loss_and_accuracy(train_losses, val_losses, train_accuracies, val_accuracies, num_epochs, work_dic)
    # 进行测试集的预测
    best_model = ConvAndAttentionModel(
        first_conv_out_channels=first_conv_out_channels,
        first_kernel_size=first_kernel_size,
        second_conv_out_channels=second_conv_out_channels,
        second_kernel_size=second_kernel_size,
        attention_output_dim=attention_output_dim,
        maxpool_kernel_size=maxpool_kernel_size
    )
    best_model.load_state_dict(torch.load(model_save_path, weights_only=True))
    best_model.to(device)
    '''
    使用我们优化后的evaluate函数，一行代码完成所有预测和结果收集
    注意：我们不需要 best_model.eval()，因为函数内部已经设置了
    我们也不需要 test_loss 和 test_acc，所以可以用 _ 来忽略它们
    '''
    _, _, test_true_labels, test_probs = evaluate(best_model, test_loader, criterion, device, return_preds=True)
    
    np.save(work_dic + f'/predict_test/test_probs{parameter}.npy', test_probs)
    np.save(work_dic + f'/predict_test/test_labels{parameter}.npy', test_true_labels)
    # 绘制 ROC&PR 曲线
    plot_roc_curve(test_true_labels, test_probs, work_dic)
    plot_pr_curve(test_true_labels, test_probs, work_dic)

    test_pred_labels = np.argmax(test_probs, axis=1)

    # 生成混淆矩阵
    cm = confusion_matrix(test_true_labels, test_pred_labels)

    # 绘制混淆矩阵
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Class 0', 'Class 1'],
                yticklabels=['Class 0', 'Class 1'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    plt.savefig(work_dic + f'picture/confusion_matrix{parameter}.pdf', format='pdf')


if __name__ == '__main__':
    main()

Epoch 1, Current LR: 0.000500
Epoch 1/200
Train Loss: 0.0867, Train Accuracy: 49.39%
Val Loss: 0.0866, Val Accuracy: 51.27%
Epoch 2, Current LR: 0.000500
Epoch 2/200
Train Loss: 0.0866, Train Accuracy: 49.52%
Val Loss: 0.0865, Val Accuracy: 57.81%
Epoch 3, Current LR: 0.000500
Epoch 3/200
Train Loss: 0.0859, Train Accuracy: 56.91%
Val Loss: 0.0852, Val Accuracy: 56.22%
Epoch 4, Current LR: 0.000500
Epoch 4/200
Train Loss: 0.0838, Train Accuracy: 61.05%
Val Loss: 0.0822, Val Accuracy: 63.29%
Epoch 5, Current LR: 0.000500
Epoch 5/200
Train Loss: 0.0809, Train Accuracy: 63.10%
Val Loss: 0.0792, Val Accuracy: 65.19%
Epoch 6, Current LR: 0.000500
Epoch 6/200
Train Loss: 0.0790, Train Accuracy: 64.67%
Val Loss: 0.0763, Val Accuracy: 65.93%
Epoch 7, Current LR: 0.000500
Epoch 7/200
Train Loss: 0.0777, Train Accuracy: 65.56%
Val Loss: 0.0764, Val Accuracy: 67.09%
Epoch 8, Current LR: 0.000500
Epoch 8/200
Train Loss: 0.0773, Train Accuracy: 66.23%
Val Loss: 0.0756, Val Accuracy: 67.09%
Epoch 9,

## 运行完上述代码之后，去/home/ykq/projects/CPSC0926/mouse/train/traindata文件夹下面检查是否有picture等文件夹，有则进行下一步

## 画模型的roc，pr以及混淆矩阵

In [123]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, confusion_matrix, accuracy_score
import seaborn as sns

def load_data(prob_path):
    """加载概率文件和标签文件"""
    label_path = prob_path.replace('probs', 'labels')
    y_prob = np.load(prob_path)
    y_true = np.load(label_path)
    # 如果输出多列概率，取正类的概率
    if y_prob.ndim > 1 and y_prob.shape[1] > 1:
        y_prob = y_prob[:, 1]
    return y_true, y_prob

def plot_roc(y_true, y_prob, color, label_text, filename):
    """计算并绘制 ROC 曲线"""
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    auc_value = roc_auc_score(y_true, y_prob)
    plt.figure(figsize=(6, 5),dpi=300)
    plt.plot(fpr, tpr, color=color, lw=2, label=f'{label_text} (AUC = {auc_value:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.title('ROC Curve', fontsize=20)
    plt.legend(loc="lower right", fontsize=12, frameon=False)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(filename, format='pdf', dpi=300, bbox_inches='tight')
    plt.show()

def plot_pr(y_true, y_prob, color, label_text, filename):
    """计算并绘制 PR 曲线"""
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    avg_precision = np.mean(precision)
    plt.figure(figsize=(6, 5),dpi=300)
    plt.plot(recall, precision, color=color, lw=2, label=f'{label_text} (AP = {avg_precision:.2f})')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('Recall', fontsize=18)
    plt.ylabel('Precision', fontsize=18)
    plt.title('PR Curve', fontsize=20)
    plt.legend(loc="lower left", fontsize=12, frameon=False)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(filename, format='pdf', dpi=300, bbox_inches='tight')
    plt.show()

def plot_confusion_matrix(y_true, y_prob, filename):
    """绘制阈值下的混淆矩阵"""
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    y_pred = (y_prob >= optimal_threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(6, 5), dpi=300)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix\n (Threshold = {optimal_threshold:.2f})', fontsize=20)
    plt.xlabel('Predicted', fontsize=18)
    plt.ylabel('Actual', fontsize=18)
    plt.tight_layout()
    plt.savefig(filename, format='pdf', dpi=300, bbox_inches='tight')
    plt.show()

# 文件路径
prob_file_old = '/home/ykq/projects/CPSC0926/mouse/train/P/traindata/predict_test/test_probsalpha5.npy'

# 加载数据
y_true_old, y_prob_old = load_data(prob_file_old)

# 绘制并保存 GM12878 valid 的 ROC 曲线
plot_roc(y_true_old, y_prob_old, color='purple', label_text='Mouse CD8+ test', filename='/home/ykq/projects/CPSC0926/fig/mouse_valid_roc.pdf')

# 绘制并保存 GM12878 valid 的 PR 曲线
plot_pr(y_true_old, y_prob_old, color='purple', label_text='Mouse CD8+ test', filename='/home/ykq/projects/CPSC0926/fig/mouse_valid_pr.pdf')

# 绘制并保存 GM12878 valid 的混淆矩阵
plot_confusion_matrix(y_true_old, y_prob_old, filename='/home/ykq/projects/CPSC0926/fig/mouse_valid_confusion_matrix.pdf')